In [ ]:
!pip install numpy Pillow --quiet

In [ ]:
import numpy as np
from PIL import Image
import random


def text_to_bits(text):
    """Convert text to a string of bits."""
    return ''.join(format(ord(char), '08b') for char in text)

def bits_to_text(bits):
    """Convert a string of bits back to text."""
    chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
    return ''.join(chr(int(char, 2)) for char in chars)

def generate_random_key(length):
    """Generate a random binary string of a given length."""
    return ''.join(random.choice('01') for _ in range(length))

def embed_lsb(image_array, data_bits, offset=0):
    """Embeds a bitstream into the LSB of an image array."""
    original_shape = image_array.shape
    flat_array = image_array.flatten()

    if (offset + len(data_bits)) > len(flat_array):
        raise ValueError("Data is too large to be embedded in the image.")

    for i in range(len(data_bits)):
        flat_array[offset + i] = (flat_array[offset + i] & 254) | int(data_bits[i])

    return flat_array.reshape(original_shape)

def extract_lsb(image_array, data_len, offset=0):
    """Extracts a bitstream of a given length from an image array's LSB."""
    flat_array = image_array.flatten()
    bits = ""
    for i in range(data_len):
        bits += str(flat_array[offset + i] & 1)
    return bits

# --- Phase 2: Quantum-Inspired Encryption (BB84 Simulation) ---

def bb84_encrypt(message_bits, key_bits):
    """
    Simulates BB84 encoding.
    Key '0' -> Rectilinear basis (+): 0 -> |0⟩, 1 -> |1⟩
    Key '1' -> Diagonal basis (x):   0 -> |+⟩, 1 -> |−⟩
    We'll represent these states with 2 bits for embedding:
    |0⟩ -> 00, |1⟩ -> 01, |+⟩ -> 10, |−⟩ -> 11
    """
    if len(message_bits) != len(key_bits):
        raise ValueError("Message and key must have the same length.")

    encrypted_payload = ""
    for i in range(len(message_bits)):
        message_bit = message_bits[i]
        basis_bit = key_bits[i]

        if basis_bit == '0':
            if message_bit == '0':
                encrypted_payload += '00'
            else:
                encrypted_payload += '01'
        else:
            if message_bit == '0':
                encrypted_payload += '10'
            else:
                encrypted_payload += '11'

    return encrypted_payload

def bb84_decrypt(encrypted_payload_bits, key_bits):
    """
    Simulates BB84 decoding.
    The receiver uses the same key (bases) to measure the 'qubits'.
    """
    if len(encrypted_payload_bits) != 2 * len(key_bits):
        raise ValueError("Encrypted payload length must be twice the key length.")

    decrypted_bits = ""
    for i in range(len(key_bits)):
        basis_bit = key_bits[i]
        qubit_representation = encrypted_payload_bits[i*2:(i*2)+2]

        if basis_bit == '0':
            if qubit_representation == '00':
                decrypted_bits += '0'
            elif qubit_representation == '01':
                decrypted_bits += '1'
            else:
                decrypted_bits += random.choice('01')

        else:
            if qubit_representation == '10':
                decrypted_bits += '0'
            elif qubit_representation == '11':
                decrypted_bits += '1'
            else:
                decrypted_bits += random.choice('01')

    return decrypted_bits

# --- Main Embedding and Extraction Functions ---

def embed_message(image_path, secret_message, output_path):
    """Main function to perform the full embedding process."""
    print("--- Starting Embedding Process ---")

    image = Image.open(image_path).convert('RGB')
    image_array = np.array(image, dtype=np.uint8)

    message_bits = text_to_bits(secret_message)
    message_len = len(message_bits)

    bits_needed = 32 + message_len + (2 * message_len)
    bits_available = image_array.size

    if bits_needed > bits_available:
        raise ValueError(f"Message is too large for this image. "
                         f"Needs {bits_needed} bits, but only {bits_available} are available.")
    print("Image capacity is sufficient.")

    print(f"Generating a key of length {message_len} for the message...")
    key_bits = generate_random_key(message_len)

    key_len_bits = format(message_len, '032b')
    data_to_embed_first = key_len_bits + key_bits

    print("Phase 1: Embedding key into the image LSBs...")
    image_array = embed_lsb(image_array, data_to_embed_first, offset=0)

    print("Phase 2: Encrypting message using BB84 simulation...")
    encrypted_payload = bb84_encrypt(message_bits, key_bits)

    print("Phase 3: Embedding encrypted payload into the image...")
    key_storage_len = len(data_to_embed_first)
    image_array = embed_lsb(image_array, encrypted_payload, offset=key_storage_len)

    stego_image = Image.fromarray(image_array)
    stego_image.save(output_path)
    print(f"Embedding complete. Stego-image saved to '{output_path}'")

def extract_message(stego_image_path):
    """Main function to perform the full extraction process."""
    print("\n--- Starting Extraction Process ---")

    stego_image = Image.open(stego_image_path).convert('RGB')
    image_array = np.array(stego_image, dtype=np.uint8)

    print("Phase 1: Extracting key length from the image...")
    key_len_bits = extract_lsb(image_array, 32, offset=0)
    key_len = int(key_len_bits, 2)

    bits_needed_for_full_message = 32 + (3 * key_len)
    bits_available = image_array.size

    if bits_needed_for_full_message > bits_available:
        raise ValueError(f"The encoded message length ({key_len}) is too large for this image size. "
                         "Are you sure this is the correct stego-image file?")

    print(f"Extracted key length is {key_len}. Proceeding to extract key...")
    key_bits = extract_lsb(image_array, key_len, offset=32)

    print("Phase 2: Extracting encrypted payload...")
    payload_offset = 32 + key_len
    encrypted_payload_len = 2 * key_len
    encrypted_payload_bits = extract_lsb(image_array, encrypted_payload_len, offset=payload_offset)

    print("Phase 3: Decrypting payload using the extracted key...")
    decrypted_message_bits = bb84_decrypt(encrypted_payload_bits, key_bits)
    secret_message = bits_to_text(decrypted_message_bits)

    print(f"Extraction complete!")
    return secret_message


if __name__ == "__main__":
    COVER_IMAGE = "cover_image.png"
    STEGO_IMAGE = "stego_image.png"
    SECRET_MESSAGE = "This is a secret message demonstrating quantum steganography"

    try:
        embed_message(COVER_IMAGE, SECRET_MESSAGE, STEGO_IMAGE)

        extracted_message = extract_message(STEGO_IMAGE)
        print("\nOriginal Message: ", SECRET_MESSAGE)
        print("Extracted Message:", extracted_message)

        if SECRET_MESSAGE == extracted_message:
            print("\nVerification successful! The message was extracted correctly. 🎉")
        else:
            print("\nVerification failed. The message was not extracted correctly. 😢")

    except FileNotFoundError:
        print(f"Error: The cover image '{COVER_IMAGE}' was not found.")
        print("Please make sure you have an image file with that name in the same directory.")
    except ValueError as e:
        print(f"Error: {e}")

--- Starting Embedding Process ---
Image capacity is sufficient.
Generating a key of length 480 for the message...
Phase 1: Embedding key into the image LSBs...
Phase 2: Encrypting message using BB84 simulation...
Phase 3: Embedding encrypted payload into the image...
Embedding complete. Stego-image saved to 'stego_image.png'

--- Starting Extraction Process ---
Phase 1: Extracting key length from the image...
Extracted key length is 480. Proceeding to extract key...
Phase 2: Extracting encrypted payload...
Phase 3: Decrypting payload using the extracted key...
Extraction complete!

Original Message:  This is a secret message demonstrating quantum steganography
Extracted Message: This is a secret message demonstrating quantum steganography

Verification successful! The message was extracted correctly. 🎉


In [ ]:
import numpy as np
from PIL import Image

def visualize_lsb(stego_image_path, output_path):
    """
    Extracts the LSB plane of an image to visualize hidden data.
    """
    print(f"Loading '{stego_image_path}' to visualize LSB data...")

    try:
        stego_image = Image.open(stego_image_path).convert('RGB')
        image_array = np.array(stego_image, dtype=np.uint8)

        lsb_visual_array = (image_array & 1) * 255

        visual_image = Image.fromarray(lsb_visual_array)
        visual_image.save(output_path)

        print(f"Visualization complete! Check the new file: '{output_path}'")

    except FileNotFoundError:
        print(f"Error: The file '{stego_image_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    STEGO_IMAGE = "stego_image.png"
    VISUALIZATION_OUTPUT = "lsb_visualization.png"

    visualize_lsb(STEGO_IMAGE, VISUALIZATION_OUTPUT)

Loading 'stego_image.png' to visualize LSB data...
Visualization complete! Check the new file: 'lsb_visualization.png'
